In [2]:
import pandas as pd
import plotly.graph_objects as go

import yfinance as yf

import warnings
warnings.filterwarnings("ignore")

import datetime as dt

In [3]:
# list of tickers to compare
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'V', 'WMT', 
           'NFLX', 'PFE', 'BABA', 'RACE', 'MC.PA',           
           '^DJI', '^GSPC', '^IXIC', '^RUT', '^N225', '^GDAXI', '^FCHI', 
           'CL=F', 'GC=F', 'SI=F',
           'BTC-USD', 'ETH-USD', 'DOGE-USD', 'BNB-USD', 'XRP-USD', 'SOL-USD', 'LTC-USD'
           ]

In [4]:
start_date = "2023-01-01"
end_date = dt.datetime.now().strftime("%Y-%m-%d")

# Download historical data from yf API
data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# download data to csv, filename have tickers joined by underscore and end_date
data.to_csv(f"assets_comparison_{end_date}.csv")

[*********************100%***********************]  31 of 31 completed

1 Failed download:
['NVDA']: YFPricesMissingError('possibly delisted; no price data found  (1d 2023-01-01 -> 2026-01-14) (Yahoo error = "Error occurred while retrieving timeseries, keys: [RedisKey [key=TF[1,YF_HIST][31988], cluster=finance]]")')


In [5]:
# Load the CSV with MultiIndex columns (Tickers, OHLCV)
df = pd.read_csv(f"assets_comparison_{end_date}.csv", header=[0,1], index_col=0)

# Drop any rows that are completely NaN (e.g. 'Date' row)
df = df.dropna(how='all')

# Convert all values to float
df = df.astype(float)

# set index as datetime
df.index = pd.to_datetime(df.index)

# keep only level 1 'Close' prices
df = df.xs('Close', level=1, axis=1)

# Show the result
df.head()

Ticker,ETH-USD,SOL-USD,XRP-USD,BABA,META,^FCHI,CL=F,V,^GDAXI,^DJI,...,TSLA,BTC-USD,MSFT,AAPL,LTC-USD,MC.PA,DOGE-USD,BNB-USD,PFE,SI=F
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-01,1200.964844,9.982173,0.338763,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,16625.080078,NaN,NaN,70.815659,NaN,0.070225,244.136978,NaN,NaN
2023-01-02,1214.656616,11.272967,0.348556,NaN,NaN,6594.569824,NaN,NaN,14069.259766,NaN,...,NaN,16688.470703,NaN,NaN,74.862907,654.875854,0.071454,245.535904,NaN,NaN
2023-01-03,1214.778809,13.344046,0.343824,87.352951,123.874695,6623.890137,76.930000,202.755203,14181.669922,33136.371094,...,108.099998,16679.857422,233.985657,123.211197,75.602722,663.084229,0.070468,246.133362,43.092449,24.059000
2023-01-04,1256.526611,13.435113,0.347794,98.692307,126.486450,6776.430176,72.839996,207.858521,14490.780273,33269.769531,...,113.639999,16863.238281,223.750336,124.482040,75.433907,696.295105,0.073237,259.119690,42.142506,23.792000
2023-01-05,1250.438599,13.419330,0.338039,99.319107,126.059441,6761.500000,73.669998,206.392059,14436.309570,32930.078125,...,110.339996,16836.736328,217.118881,123.161942,74.250809,687.237488,0.071550,256.422852,41.747379,23.257999


In [6]:
# date filter
# keep only last year of data
one_year_ago = dt.datetime.now() - dt.timedelta(days=365)
df = df[df.index >= one_year_ago]

In [7]:
# df copies for last 6 months and 3 months of data
df_6m = df[df.index >= (dt.datetime.now() - dt.timedelta(days=182))]
df_3m = df[df.index >= (dt.datetime.now() - dt.timedelta(days=91))]
df_1m = df[df.index >= (dt.datetime.now() - dt.timedelta(days=30))]
df_1w = df[df.index >= (dt.datetime.now() - dt.timedelta(days=7))]

In [8]:
# calculate simple returns with pct_change()
simple_returns = df.pct_change().fillna(0)

# cumulative product of simple returns (correct for compounding)
cumprod_simple = (1 + simple_returns).cumprod() - 1

In [9]:
# calculate simple returns with pct_change() for all timeframes
simple_returns_6m = df_6m.pct_change().fillna(0)
simple_returns_3m = df_3m.pct_change().fillna(0)
simple_returns_1m = df_1m.pct_change().fillna(0)
simple_returns_1w = df_1w.pct_change().fillna(0)

# cumulative product of simple returns (correct for compounding) for all timeframes
cumprod_simple_6m = (1 + simple_returns_6m).cumprod() - 1
cumprod_simple_3m = (1 + simple_returns_3m).cumprod() - 1
cumprod_simple_1m = (1 + simple_returns_1m).cumprod() - 1
cumprod_simple_1w = (1 + simple_returns_1w).cumprod() - 1

In [10]:
# plot cumprod_simple
fig = go.Figure()
for column in cumprod_simple.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple.index,
            y=cumprod_simple[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="All Tickers - Cumulative Simple Returns (Last Year)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [11]:
# Bar plot of final cumulative returns for each ticker
final_returns = cumprod_simple.iloc[-1].dropna().sort_values()

# Create colors based on positive/negative returns
colors = ['green' if x >= 0 else 'red' for x in final_returns.values]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=final_returns.values,
        y=final_returns.index,
        orientation='h',
        marker_color=colors,
        text=[f"{x:.1%}" for x in final_returns.values],
        textposition='outside',
    )
)

fig.update_layout(
    title="All Tickers - Cumulative Returns (Last Year)",
    xaxis_title="Cumulative Return",
    yaxis_title="Ticker",
    template="plotly_white",
    xaxis_tickformat=".0%",
    height=max(500, len(final_returns) * 20),
    margin=dict(l=200, r=80, t=80, b=60),
)
fig.show()

In [12]:
# plot last 6 months of cumprod_simple_6m returns 
fig = go.Figure()
for column in cumprod_simple_6m.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_6m.index,
            y=cumprod_simple_6m[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="All Tickers - Cumulative Simple Returns (Last 6 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [13]:
# Bar plot of final cumulative returns for each ticker (Last 6 Months)
final_returns_6m = cumprod_simple_6m.iloc[-1].dropna().sort_values()

# Create colors based on positive/negative returns
colors = ['green' if x >= 0 else 'red' for x in final_returns_6m.values]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=final_returns_6m.values,
        y=final_returns_6m.index,
        orientation='h',
        marker_color=colors,
        text=[f"{x:.1%}" for x in final_returns_6m.values],
        textposition='outside',
    )
)

fig.update_layout(
    title="All Tickers - Cumulative Returns (Last 6 Months)",
    xaxis_title="Cumulative Return",
    yaxis_title="Ticker",
    template="plotly_white",
    xaxis_tickformat=".0%",
    height=max(500, len(final_returns_6m) * 20),
    margin=dict(l=200, r=80, t=80, b=60),
)
fig.show()

In [14]:
# plot last 3 months of cumprod_simple_3m returns 
fig = go.Figure()
for column in cumprod_simple_3m.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_3m.index,
            y=cumprod_simple_3m[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="All Tickers - Cumulative Simple Returns (Last 3 Months)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [15]:
# Bar plot of final cumulative returns for each ticker (Last 3 Months)
final_returns_3m = cumprod_simple_3m.iloc[-1].dropna().sort_values()

# Create colors based on positive/negative returns
colors = ['green' if x >= 0 else 'red' for x in final_returns_3m.values]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=final_returns_3m.values,
        y=final_returns_3m.index,
        orientation='h',
        marker_color=colors,
        text=[f"{x:.1%}" for x in final_returns_3m.values],
        textposition='outside',
    )
)

fig.update_layout(
    title="All Tickers - Cumulative Returns (Last 3 Months)",
    xaxis_title="Cumulative Return",
    yaxis_title="Ticker",
    template="plotly_white",
    xaxis_tickformat=".0%",
    height=max(500, len(final_returns_3m) * 20),
    margin=dict(l=200, r=80, t=80, b=60),
)
fig.show()

In [16]:
# plot last 1 month of cumprod_simple_1m returns 
fig = go.Figure()
for column in cumprod_simple_1m.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_1m.index,
            y=cumprod_simple_1m[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="All Tickers - Cumulative Simple Returns (Last Month)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [17]:
# Bar plot of final cumulative returns for each ticker (Last Month)
final_returns_1m = cumprod_simple_1m.iloc[-1].dropna().sort_values()

# Create colors based on positive/negative returns
colors = ['green' if x >= 0 else 'red' for x in final_returns_1m.values]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=final_returns_1m.values,
        y=final_returns_1m.index,
        orientation='h',
        marker_color=colors,
        text=[f"{x:.1%}" for x in final_returns_1m.values],
        textposition='outside',
    )
)

fig.update_layout(
    title="All Tickers - Cumulative Returns (Last Month)",
    xaxis_title="Cumulative Return",
    yaxis_title="Ticker",
    template="plotly_white",
    xaxis_tickformat=".0%",
    height=max(500, len(final_returns_1m) * 20),
    margin=dict(l=200, r=80, t=80, b=60),
)
fig.show()

In [18]:
# plot last 1 week of cumprod_simple_1w returns 
fig = go.Figure()
for column in cumprod_simple_1w.columns:
    fig.add_trace(
        go.Scatter(
            x=cumprod_simple_1w.index,
            y=cumprod_simple_1w[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title="All Tickers - Cumulative Simple Returns (Last Week)",
    xaxis_title="Date",
    yaxis_title="Cumulative Return",
    legend=dict(orientation="h", yanchor="top", y=-0.15, xanchor="center", x=0.5),
    template="plotly_white",
    margin=dict(t=80, b=120),
)
fig.show()

In [19]:
# Bar plot of final cumulative returns for each ticker (Last Week)
final_returns_1w = cumprod_simple_1w.iloc[-1].dropna().sort_values()

# Create colors based on positive/negative returns
colors = ['green' if x >= 0 else 'red' for x in final_returns_1w.values]

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=final_returns_1w.values,
        y=final_returns_1w.index,
        orientation='h',
        marker_color=colors,
        text=[f"{x:.1%}" for x in final_returns_1w.values],
        textposition='outside',
    )
)

fig.update_layout(
    title="All Tickers - Cumulative Returns (Last Week)",
    xaxis_title="Cumulative Return",
    yaxis_title="Ticker",
    template="plotly_white",
    xaxis_tickformat=".0%",
    height=max(500, len(final_returns_1w) * 20),
    margin=dict(l=200, r=80, t=80, b=60),
)
fig.show()